Here is my code to find angular separation

In [11]:
#import packages 
import numpy as np
import pandas as pd
import csv
import sys
import matplotlib.pyplot as plt
from astropy import units as u  
import time

In [20]:
galaxiesFull = pd.read_csv("galaxies_full.csv", sep = ',')  
ROSAT = pd.read_csv("2rxs_lite.txt", delim_whitespace=True)  

RA_Galaxies = ROSAT.iloc[:, 2] #right ascention
DEC_Galaxies = ROSAT.iloc[:, 3] #decllination
PosErr_ROSAT = ROSAT.iloc[:, 4] #position error

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
PosErr_ROSAT = ROSAT.iloc[:, 4] #position error

In [21]:
start = time.time() #start a time to see how long it takes


count = 0
theta = [] #the angular separation
d1 = []
d2 = []
ra1 = []
ra2 = []
snr1 = []
snr2 = []
offset = [] 
k = 0


for i in range(np.size(DEC_Galaxies)): 
        
        #tmp = np.where( ((np.absolute(DEC_Galaxies[i] - DEC_ROSAT) < 1)) & ( (np.absolute(RA_Galaxies[i]-RA_ROSAT[i]) * np.cos(DEC_ROSAT*(np.pi/180)) )  < 1 ) )  #conditions it need to meet
        #print(type(tmp))
        tmp1 = np.where( np.absolute(DEC_Galaxies[i] - DEC_ROSAT) < 1) 
        tmp2 = np.where( np.absolute(RA_Galaxies[i]-RA_ROSAT) * np.cos(DEC_ROSAT*(np.pi/180))  < 1 ) 
        resultArray = (np.intersect1d(tmp1, tmp2))
        
        #print(np.size(resultArray))
        if np.size(resultArray) ==0: 
                continue
        else: 
                for j in resultArray:
                #tmp[0]:
                        #j = tmp[0]
                        #print(j)
                        count+= 1
                        theta_rad = np.sin(DEC_Galaxies[i]*np.pi/180)* np.sin(DEC_ROSAT[j]*np.pi/180) + np.cos(DEC_Galaxies[i]*np.pi/180)*np.cos(DEC_ROSAT[j]*np.pi/180)*np.cos(np.absolute(RA_Galaxies[i]-RA_ROSAT[j])*np.pi/180) #returns answer in rad
                        #print(theta_rad)
                        theta_arcsec = np.arccos(theta_rad) * 206265
                        #print(theta_arcsec)
                        #print(type(theta_arcsec))
                        if theta_arcsec < 360: 
                
                                poserr1 = 1
                                if PosErr_ROSAT[j] < 6:
                                        poserr2 = 6

                                else:
                                        poserr2 = PosErr_ROSAT[j] 
                                confid = theta_arcsec/np.sqrt(poserr1**2 + poserr2**2) #finding theta hat
                                #print(theta_arcsec)
                                if confid < 4:
                                        theta.append(theta_arcsec)
                                        offset.append(confid)
                                        d1.append(DEC_Galaxies[i])
                                        d2.append(DEC_ROSAT[j])
                                        ra1.append(RA_Galaxies[i])
                                        ra2.append(RA_ROSAT[j])
                                        snr1.append(poserr1)
                                        snr2.append(poserr2)
                                        #print(theta)
                                        #tmp_num = theta_arcsec
                                        k += 1

                
print(count)
print(k)
print(np.size(theta))
#a = np.sin(90*np.pi/180)
#print(a)

#ang_sep = np.zeros(count)
#for i in range(count):
    #ang_sep[i] = round(theta[i].value, 3)
#    ang_sep[i] = theta[0][i]

#print(np.shape(tmp))
print("Writing file")

with open(r'galaxies_ROSAT.txt', 'w') as fp:
        fp.write("Theta\tOffset\tRA-SDSS\tDEC-SDSS\tPosErr-SDSS\tRA-ROSAT\tDEC-ROSAT\tPosErr-ROSAT\n")
        for i in range(k):
                fp.write("%4.1f\t%4.2f\t%12.6f\t%12.6f\t%6.2f\t%12.6f\t%12.6f\t%6.2f\n" %((theta[i]), (offset[i]), (ra1[i]), (d1[i]), (snr1[i]), (ra2[i]), (d2[i]),   (snr2[i]) ))
                #fp.write("%4.1f\t%4.2f\t%f\t%f\t%6.2f\t%f\t%f\t%6.2f\n" %((theta[i]), (offset[i]), (ra1[i]), (d1[i]), (snr1[i]), (ra2[i]), (d2[i]),   (snr2[i]) ))

print(f'Numpy: {time.time() - start} seconds')

/var/folders/3f/g9qgz71j35z23v4byb9jtltc00031c/T/ipykernel_21873/2094083474.py:34: RuntimeWarning: invalid value encountered in arccos
  theta_arcsec = np.arccos(theta_rad) * 206265


1954863
108183
108183
Writing file
Numpy: 359.896940946579 seconds


In [22]:
np.size(RA_Galaxies)

113314

There are 11314 galaxies and it seems thaat 108183 of them are the exact source in this list from SDSS


Now I will try again with SDSS.

In [23]:
x

NameError: name 'x' is not defined

In [28]:
start = time.time() #start a time to see how long it takes


Galaxies = pd.read_csv("galaxies_full.csv", sep = ',')  
RA_Galaxies = Galaxies.iloc[:, 2] #right ascention
DEC_Galaxies = Galaxies.iloc[:, 3] #decllination
PosErr_Galaxies = Galaxies.iloc[:, 4] #position error


SDSS = pd.read_csv("dr7_sara.csv", sep=',', header = 1) #SDSS
RA_SDSS = SDSS.iloc[:, 0] #right ascention
DEC_SDSS = SDSS.iloc[:, 1] #decllination
PosErr_SDSS = SDSS.iloc[:, 2] #signal to noise ratio

print("RA SDSS", np.size(DEC_SDSS))
print("DEC SDSS", np.size(RA_SDSS))
print("RA Galaxies", np.size(RA_Galaxies))
print("DEC Galaxies", np.size(DEC_Galaxies))

count = 0

theta = [] #the angular separation
d1 = []
d2 = []
ra1 = []
ra2 = []
PosErr1 = []
PosErr2 = []
offset = []
k = 0

for i in range(np.size(DEC_SDSS)): 
        
        #tmp = np.where( ((np.absolute(DEC_SDSS[i] - DEC_Galaxies) < 1)) & ( (np.absolute(RA_SDSS[i]-RA_Galaxies[i]) * np.cos(DEC_Galaxies*(np.pi/180)) )  < 1 ) )  #conditions it need to meet
        #print(type(tmp))
        tmp1 = np.where( np.absolute(DEC_SDSS[i] - DEC_Galaxies) < 1) 
        tmp2 = np.where( np.absolute(RA_SDSS[i]-RA_Galaxies) * np.cos(DEC_Galaxies*(np.pi/180))  < 1 ) 
        resultArray = (np.intersect1d(tmp1, tmp2))
        
        #print(np.size(resultArray))
        if np.size(resultArray) ==0: 
                continue
        else: 
                for j in resultArray:
                #tmp[0]:
                        #j = tmp[0]
                        #print(j)
                        count+= 1
                        theta_rad = np.sin(DEC_SDSS[i]*np.pi/180)* np.sin(DEC_Galaxies[j]*np.pi/180) + np.cos(DEC_SDSS[i]*np.pi/180)*np.cos(DEC_Galaxies[j]*np.pi/180)*np.cos(np.absolute(RA_SDSS[i]-RA_Galaxies[j])*np.pi/180) #returns answer in rad
                        #print(theta_rad)
                        theta_arcsec = np.arccos(theta_rad) * 206265
                        #print(theta_arcsec)
                        #print(type(theta_arcsec))
                        if theta_arcsec < 360: 

                                poserr1 = 6
                                if PosErr_Galaxies[j] < 6:
                                        poserr2 = 6

                                else:
                                        poserr2 = PosErr_Galaxies[j] 
                                confid = theta_arcsec/np.sqrt(poserr1**2 + poserr2**2) #finding theta hat
                                #print(theta_arcsec)
                                if confid < 4:
                                        theta.append(theta_arcsec)
                                        offset.append(confid)
                                        d1.append(DEC_SDSS[i])
                                        d2.append(DEC_Galaxies[j])
                                        ra1.append(RA_SDSS[i])
                                        ra2.append(RA_Galaxies[j])
                                        PosErr1.append(poserr1)
                                        PosErr2.append(poserr2)
                                        #print(theta)
                                        #tmp_num = theta_arcsec
                                        k += 1

                
print(count)
print(k)
print(np.size(theta))
#a = np.sin(90*np.pi/180)
#print(a)

ang_sep = np.zeros(count)
#for i in range(count):
    #ang_sep[i] = round(theta[i].value, 3)
#    ang_sep[i] = theta[0][i]

#print(np.shape(tmp))
print("Writing file")

with open(r'dr7_galaxies.txt', 'w') as fp:
        fp.write("Theta\tOffset\tRA-SDSS\tDEC-SDSS\tPosErr-SDSS\tRA-Galaxies\tDEC-Galaxies\tPosErr-Galaxies\n")
        for i in range(k):
                fp.write("%4.1f\t%4.2f\t%12.6f\t%12.6f\t%6.2f\t%12.6f\t%12.6f\t%6.2f\n" %((theta[i]), (offset[i]), (ra1[i]), (d1[i]), (PosErr1[i]), (ra2[i]), (d2[i]),   (PosErr2[i]) ))
                #fp.write("%4.1f\t%4.2f\t%f\t%f\t%6.2f\t%f\t%f\t%6.2f\n" %((theta[i]), (offset[i]), (ra1[i]), (d1[i]), (PosErr1[i]), (ra2[i]), (d2[i]),   (PosErr2[i]) ))

print(f'Numpy: {time.time() - start} seconds')

RA SDSS 9525
DEC SDSS 9525
RA Galaxies 1573
DEC Galaxies 1573
0
0
0
Writing file
Numpy: 8.961338758468628 seconds
